In [1]:
import sys
import os
import melt
import gezi
import tensorflow as tf
import numpy as np

tensorflow_version: 1.14.0
torch_version: 1.2.0
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [54]:
pb_path = '/home/gezi/tmp/rank/data/debug/2019102407_1-2019102404_24/model.pb'

In [3]:
predictor = melt.Predictor()
print('model.pb:', pb_path, file=sys.stderr)
predictor.load_graph(pb_path, 'prefix', frozen_map_file=pb_path.replace('.pb', '.map'))
print('load pb done', file=sys.stderr)

model.pb: /home/gezi/tmp/rank/data/debug/2019102407_1-2019102404_24/model.pb
------------frozen_graph_name: prefix
load pb done


In [4]:
predictor.graph.get_collection('index_feed')

[<tf.Tensor 'prefix/index_feed:0' shape=(?, ?) dtype=int64>]

In [36]:
fnames = open('/home/gezi/tmp/rank/data/debug/2019102407_1-2019102404_24/feat_fields.txt').readlines()
fnames = ['None'] + [x.split()[0] for x in fnames]

In [55]:
debug_file = '/home/gezi/tmp/rank/data/debug/2019102407_1-2019102404_24/debug.txt'

In [56]:
def filter(index, value, field, masked_fields):
    flags = np.asarray([True if x not in masked_fields else False for x in field])
    index, value, field = np.asarray(index), np.asarray(value), np.asarray(field)
    index = list(index[flags])
    value = list(value[flags])
    field = list(field[flags])
    return index, value, field

In [59]:
for line in open(debug_file):
  for i in range(1, 96):
      mid, docid, duration, indexes, fields, values = line.strip().split('\t')
      index = list(map(int, indexes.split()))
      value = list(map(float, values.split()))
      field = list(map(int, fields.split()))
      index, value, field = filter(index, value, field, [i])
#       print(index)
#       print(value)
#       print(field)
      feed_dict = {predictor.graph.get_collection('index_feed')[0]: [index],
                   predictor.graph.get_collection('value_feed')[0]: [value],
                   predictor.graph.get_collection('field_feed')[0]: [field]}
      pred = predictor.predict('pred', feed_dict)
      pred = pred[0][0]
      print(i, fnames[i], mid, docid, duration, pred, sep='\t')

1	AGE	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
2	CAGATTP	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
3	CEDATTP	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
4	CITATCOTP	01d9867474035026342	19a22n0l4lySEy	0	0.2588722
5	CRWATIDQUALS	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
6	CSXATTP	01d9867474035026342	19a22n0l4lySEy	0	0.2788722
7	DATCMT	01d9867474035026342	19a22n0l4lySEy	0	0.2588722
8	DATCMTLIKE	01d9867474035026342	19a22n0l4lySEy	0	0.2788722
9	DATCMTRPY	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
10	DATCTR	01d9867474035026342	19a22n0l4lySEy	0	0.3188722
11	DATRD	01d9867474035026342	19a22n0l4lySEy	0	0.2988722
12	DPTCT	01d9867474035026342	19a22n0l4lySEy	0	0.24887219
13	EDU	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
14	FAVORNUM	01d9867474035026342	19a22n0l4lySEy	0	0.2988722
15	IACID	01d9867474035026342	19a22n0l4lySEy	0	0.2888722
16	IATAC	01d9867474035026342	19a22n0l4lySEy	0	0.2988722
17	IATKW	01d9867474035026342	19a22n0l4lySEy	0	0.2788722
18	IATKWSE	01d986747403502634

In [58]:
for line in open(debug_file):
  mid, docid, duration, indexes, fields, values = line.strip().split('\t')
  index = list(map(int, indexes.split()))
  value = list(map(float, values.split()))
  field = list(map(int, fields.split()))
#   index, value, field = filter(index, value, field, [73])
#       print(index)
#       print(value)
#       print(field)
  feed_dict = {predictor.graph.get_collection('index_feed')[0]: [index],
               predictor.graph.get_collection('value_feed')[0]: [value],
               predictor.graph.get_collection('field_feed')[0]: [field]}
  pred = predictor.predict('pred', feed_dict)
  pred = pred[0][0]
  print(mid, docid, duration, pred, sep='\t')

01d9867474035026342	19a22n0l4lySEy	0	0.2888732
